<a href="https://colab.research.google.com/github/rusamentiaga/pytorch-test/blob/main/3_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [2]:
device

device(type='cuda')

In [ ]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [3]:
!ls data/MNIST/processed/

ls: cannot access 'data/MNIST/processed/': No such file or directory


In [11]:


# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Epoch [1/5], Step [100/600], Loss: 0.2347
Epoch [1/5], Step [200/600], Loss: 0.1861
Epoch [1/5], Step [300/600], Loss: 0.1745
Epoch [1/5], Step [400/600], Loss: 0.2608
Epoch [1/5], Step [500/600], Loss: 0.1049
Epoch [1/5], Step [600/600], Loss: 0.1286
Epoch [2/5], Step [100/600], Loss: 0.0893
Epoch [2/5], Step [200/600], Loss: 0.0541
Epoch [2/5], Step [300/600], Loss: 0.2584
Epoch [2/5], Step [400/600], Loss: 0.1124
Epoch [2/5], Step [500/600], Loss: 0.3023
Epoch [2/5], Step [600/600], Loss: 0.1079
Epoch [3/5], Step [100/600], Loss: 0.1280
Epoch [3/5], Step [200/600], Loss: 0.0959
Epoch [3/5], Step [300/600], Loss: 0.0526
Epoch [3/5], Step [400/600], Loss: 0.0253
Epoch [3/5], Step [500/600], Loss: 0.0506
Epoch [3/5], Step [600/600], Loss: 0.1078
Epoch [4/5], Step [100/600], Loss: 0.1062
Epoch [4/5], Step [200/600], Loss: 0.0256
Epoch [4/5], Step [300/600], Loss: 0.0510
Epoch [4/5], Step [400/600], Loss: 0.0405
Epoch [4/5], Step [500/600], Loss: 0.0474
Epoch [4/5], Step [600/600], Loss: